In [10]:
# !pip install tweepy
# !pip install kafka-python

In [3]:
# # # # # Twitter API Libraries # # # #
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

# # # # # KAFKA Libraries # # # #
from kafka import KafkaProducer

import os

In [4]:
CONSUMER_KEY = os.environ['API_KEY']
CONSUMER_SECRET = os.environ['API_SECRET_KEY']
ACCESS_TOKEN = os.environ['ACCESS_TOKEN']
ACCESS_TOKEN_SECRET = os.environ['ACCESS_TOKEN_SECRET']

In [6]:
class TStreamer:
    """
    Class for streaming and processing live tweets.
    """
    def __init__(self, t_topic, t_producer):
        self.topic = t_topic
        self.producer = t_producer

    def stream_tweets(self, hash_tag_list):
        # This handles Twitter authetification and the connection to Twitter Streaming API
        listener = Listener(self.topic, self.producer)
        auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
        auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
        stream = Stream(auth, listener)

        # This line filter Twitter Streams to capture data by the keywords:
        stream.filter(track=hash_tag_list)

In [7]:
class Listener(StreamListener):
    """
    This is a basic listener that just prints received tweets to stdout.
    """
    def __init__(self, topic: str, producer: KafkaProducer ):
        self.topic = topic if topic else 'topic_one'
        self.k_producer = producer

    def on_data(self, data):
        self.k_producer.send(self.topic, value=data)
        return True

    def on_error(self, status):
        print(status)

In [5]:
my_topic = "topic_one"
my_producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                                        value_serializer=lambda x: x.encode('utf-8'))

In [15]:
hash_tag_list = ["corona", "virus"]

t_streamer = TStreamer(my_topic, my_producer)
t_streamer.stream_tweets(hash_tag_list)